<a href="https://colab.research.google.com/github/Riturao/fyp/blob/master/final1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install tensorflow==1.14

     |████████████████████████████████| 109.2MB 49kB/s 
     |████████████████████████████████| 3.2MB 33.0MB/s 
     |████████████████████████████████| 491kB 48.8MB/s 
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorflow 2.2.0rc2
    Uninstalling tensorflow-2.2.0rc2:
      Successfully uninstalled tensorflow-2.2.0rc2


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
#import ost as mi
import re
import numpy as np
import pandas as pd
#from chart_studio import plotly as py
#import plotly.figure_factory as ff
from scipy import stats

import gensim
import json


import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

import itertools

import keras

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, Activation, GlobalAveragePooling1D, Flatten, Concatenate, Conv1D, MaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import concatenate
from keras.optimizers import SGD, RMSprop, Adagrad, Adam
from keras.preprocessing.text import one_hot, text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils.vis_utils import plot_model

import fnmatch

import warnings

import string
from pathlib import Path
from random import shuffle
from ast import literal_eval

warnings.filterwarnings('ignore')



In [0]:
wordnet_lemmatizer = WordNetLemmatizer()

In [0]:
WINDOWS_SIZE = 10
labels=['none','mild','moderate','moderately severe', 'severe']
num_classes = len(labels)

In [0]:
def plot_acc(history, title="Model Accuracy"):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(title)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()
    
def plot_loss(history, title="Model Loss"):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(title)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper right')
    plt.show()
    
def plot_compare_losses(history1, history2, name1="Red 1", name2="Red 2", title="Graph title"):
    plt.plot(history1.history['loss'], color="green")
    plt.plot(history1.history['val_loss'], 'r--', color="green")
    plt.plot(history2.history['loss'], color="blue")
    plt.plot(history2.history['val_loss'], 'r--', color="blue")
    plt.title(title)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train ' + name1, 'Val ' + name1, 
                'Train ' + name2, 'Val ' + name2],
               loc='upper right')
    plt.show()
    
def plot_compare_accs(history1, history2, name1="Red 1",
                      name2="Red 2", title="Graph title"):
    """Compara accuracies de dos entrenamientos con nombres name1 y name2"""
    plt.plot(history1.history['acc'], color="green")
    plt.plot(history1.history['val_acc'], 'r--', color="green")
    plt.plot(history2.history['acc'], color="blue")
    plt.plot(history2.history['val_acc'], 'r--', color="blue")
    plt.title(title)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train ' + name1, 'Val ' + name1, 
                'Train ' + name2, 'Val ' + name2], 
               loc='lower right')
    plt.show()

def plot_compare_multiple_metrics(history_array, names, colors, title="Graph title", metric='acc'):  
    legend = []
    for i in range(0, len(history_array)):
        plt.plot(history_array[i].history[metric], color=colors[i])
        plt.plot(history_array[i].history['val_' + metric], 'r--', color=colors[i])
        legend.append('Train ' + names[i])
        legend.append('Val ' + names[i])
    
    plt.title(title)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')   
    plt.axis
    plt.legend(legend, 
               loc='lower right')
    plt.show()

In [0]:
def transcripts_to_dataframe(directory):
    rows_list = []
        
    filenames = os.listdir(directory)
    
    if ".DS_Store" in filenames:
        filenames.remove(".DS_Store")
        
    for filename in filenames:
        
        transcript_path = os.path.join(directory, filename)
        transcript = pd.read_csv(transcript_path , sep='\t')
        #print(transcript)
        m = re.search("(\d{3})_TRANSCRIPT.csv", filename)
        if m:
            print(filename)
            person_id = m.group(1)
            #print(person_id)
            p = {}
            question = ""
            answer = ""
            lines = len(transcript)
            #print(lines)
            for i in range(0, lines):
                #print("entered")
                row = transcript.iloc[i]
                #print(row)
                if (row["speaker"] == "Ellie") or (i == lines - 1):
                    p["personId"] = person_id
                    #print(p)
                    if "(" in str(question):
                        question = question[question.index("(") + 1:question.index(")")]
                    p["question"] = question
                    p["answer"] = answer
                    #print(answer)
                    #print(p)
                    if question != "":
                        rows_list.append(p)
                        #print(rows_list())
                    p = {}
                    answer = ""
                    question = row["value"]
                else:
                    answer = str(answer) + " " + str(row["value"])
                    #print(1)
                #print(answer)

    all_participants = pd.DataFrame(rows_list, columns=['personId', 'question', 'answer'])
    #all_participants = pd.DataFrame(*p, columns=['personId', 'question', 'answer'])
    all_participants.to_csv(directory + 'all/all.csv', sep=',')
    print("File was created")
    return all_participants

In [0]:
from google.colab import files
uploaded = files.upload()

Saving all.csv to all (1).csv
Saving ds_total.csv to ds_total (1).csv
Saving phrases_lp.csv to phrases_lp (1).csv
Saving phrases_lp_stop.csv to phrases_lp_stop (1).csv


In [0]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

User uploaded file "all.csv" with length 2007054 bytes
User uploaded file "ds_total.csv" with length 7638 bytes
User uploaded file "phrases_lp.csv" with length 20903991 bytes
User uploaded file "phrases_lp_stop.csv" with length 37898874 bytes


In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
from google.colab import files
uploaded = files.upload()

Saving dev_split_Depression_AVEC2017.csv to dev_split_Depression_AVEC2017.csv
Saving full_test_split.csv to full_test_split.csv
Saving train_split_Depression_AVEC2017.csv to train_split_Depression_AVEC2017.csv


In [0]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

User uploaded file "dev_split_Depression_AVEC2017.csv" with length 1112 bytes
User uploaded file "full_test_split.csv" with length 620 bytes
User uploaded file "train_split_Depression_AVEC2017.csv" with length 3074 bytes


In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [0]:
#data_path = r"C:/Users/Admin/Desktop/transcripts/"
#transcripts_to_dataframe(data_path) 
all_participants = pd.read_csv( 'all.csv', sep=',')
#all_participants= pd.read_csv(data_path + '*_TRANSCRIPT.csv', sep=',')
#all_participants = pd.concat([pd.read_csv(f) for f in glob.glob('*_TRANSCRIPT.csv')], ignore_index = True)#

In [0]:
all_participants.tail() 

,Unnamed: 0,personId,question,answer
14968,14968,492,mm,NaN
14969,14969,492,what are you most proud of in your life,um i am very proud of the fact that uh i don'...
14970,14970,492,okay i think i have asked everything i need to,NaN
14971,14971,492,thanks for sharing your thoughts with me,NaN
14972,14972,492,goodbye,bye and thank you


In [0]:
all_participants.columns =  ['index','personId', 'question', 'answer']
all_participants = all_participants.astype({"index": int, "personId": float, "question": str, "answer": str })

In [0]:
#all_participants = pd.DataFrame(columns=['index','personId', 'question', 'answer'])
all_participants.columns =  ['index','personId', 'question', 'answer']
all_participants = all_participants.astype({"index": int, "personId": float, "question": str, "answer": str })

In [0]:
def text_to_wordlist(text, remove_stopwords=True, stem_words=False):    
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [wordnet_lemmatizer.lemmatize(w) for w in text if not w in stops ]
        text = [w for w in text if w != "nan" ]
    else:
        text = [wordnet_lemmatizer.lemmatize(w) for w in text]
        text = [w for w in text if w != "nan" ]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    
    text = re.sub(r"\<", " ", text)
    text = re.sub(r"\>", " ", text)
    
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

In [0]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
all_participants_mix = all_participants.copy()
all_participants_mix['answer'] = all_participants_mix.apply(lambda row: text_to_wordlist(row.answer).split(), axis=1)
all_participants_mix.head()

,index,personId,question,answer
0,0,300.0,hi i'm ellie thanks for coming in today,[]
1,1,300.0,i was created to talk to people in a safe and ...,[]
2,2,300.0,think of me as a friend i don't judge i can't ...,[]
3,3,300.0,i'm here to learn about people and would love ...,[]
4,4,300.0,i'll ask a few questions to get us started and...,[]


In [0]:
print(all_participants_mix['answer'])

0                                 []
1                                 []
2                                 []
3                                 []
4                                 []
                    ...             
14968                             []
14969    [um, proud, fact, uh, give]
14970                             []
14971                             []
14972                   [bye, thank]
Name: answer, Length: 14973, dtype: object


In [0]:
all_participants_mix_stopwords = all_participants.copy()
all_participants_mix_stopwords['answer'] = all_participants_mix_stopwords.apply(lambda row: text_to_wordlist(row.answer, remove_stopwords=False).split(), axis=1)

In [0]:
words = [w for w in all_participants_mix['answer'].tolist()]
words = set(itertools.chain(*words))
vocab_size = len(words)
print(vocab_size)

7373


In [0]:
words_stop = [w for w in all_participants_mix_stopwords['answer'].tolist()]
words_stop = set(itertools.chain(*words_stop))
vocab_size_stop = len(words_stop)
print(vocab_size_stop)

7449


In [0]:
windows_size = WINDOWS_SIZE
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(all_participants_mix['answer'])
tokenizer.fit_on_sequences(all_participants_mix['answer'])

all_participants_mix['t_answer'] = tokenizer.texts_to_sequences(all_participants_mix['answer'])
all_participants_mix['answer'].head()

0    []
1    []
2    []
3    []
4    []
Name: answer, dtype: object

In [0]:
windows_size = WINDOWS_SIZE
tokenizer = Tokenizer(num_words=vocab_size_stop)
tokenizer.fit_on_texts(all_participants_mix_stopwords['answer'])
tokenizer.fit_on_sequences(all_participants_mix_stopwords['answer'])

all_participants_mix_stopwords['t_answer'] = tokenizer.texts_to_sequences(all_participants_mix_stopwords['answer'])
all_participants_mix_stopwords.head()

,index,personId,question,answer,t_answer
0,0,300.0,hi i'm ellie thanks for coming in today,[],[]
1,1,300.0,i was created to talk to people in a safe and ...,[],[]
2,2,300.0,think of me as a friend i don't judge i can't ...,[],[]
3,3,300.0,i'm here to learn about people and would love ...,[],[]
4,4,300.0,i'll ask a few questions to get us started and...,[],[]


In [0]:
word_index = tokenizer.word_index
word_size = len(word_index)
print(word_index["sad"])

652


In [0]:
all_participants_mix_stopwords.head()

,index,personId,question,answer,t_answer
0,0,300.0,hi i'm ellie thanks for coming in today,[],[]
1,1,300.0,i was created to talk to people in a safe and ...,[],[]
2,2,300.0,think of me as a friend i don't judge i can't ...,[],[]
3,3,300.0,i'm here to learn about people and would love ...,[],[]
4,4,300.0,i'll ask a few questions to get us started and...,[],[]


In [0]:
windows_size = WINDOWS_SIZE
cont = 0
word_index = tokenizer
phrases_lp_stop = pd.DataFrame(columns=['personId','answer', 't_answer'])
answers = all_participants_mix_stopwords.groupby('personId').agg({'answer':'sum','t_answer':'sum'})

#answers = all_participants_mix_stopwords.groupby('personId').agg({' ':"sum"},axis="columns")

#answers = all_participants_mix_stopwords.groupby('personId',axis=0).aggregate('sum')
#answers1 = all_participants_mix_stopwords.groupby('personId',axis=0)

#answers= answers1.DataFrame.aggregate('sum', axis=1)
#print(answers["answer"])
print(phrases_lp_stop.head())
#print(answers[1])
#print(answers)
#answers.head()
#answers1.head()

In [0]:
print(answers['answer'])

personId
300.0    [good, atlanta, georgia, um, my, parent, are, ...
301.0    [thank, you, mmm, k, i, am, doing, good, thank...
302.0    [i, am, fine, how, about, yourself, i, am, fro...
303.0    [okay, how, bout, yourself, here, in, californ...
304.0    [i, am, doing, good, um, from, los, angeles, c...
                               ...                        
488.0    [yes, fine, oh, san, fernando, valley, uh, wel...
489.0    [yes, i, am, doing, well, thank, you, san, lui...
490.0    [yeah, i, am, doing, already, how, are, you, d...
491.0    [yes, huh, overwhelmed, i, have, a, funeral, t...
492.0    [yes, doing, pretty, good, thank, you, marylan...
Name: answer, Length: 186, dtype: object


In [0]:
cont = 0
for p in answers.iterrows():      
    words = p[1]["answer"]
    size = len(words)
    word_tokens = p[1]["t_answer"]
    print(cont)
    for i in range(size):
        sentence = words[i:min(i+windows_size,size)]  
        tokens = word_tokens[i:min(i+windows_size,size)]  
        phrases_lp_stop.loc[cont] = [p[0], sentence, tokens]
        cont = cont + 1

0
366
1910
2537
4564
5586
9031
10645
13317
14253
14978
16167
16840
18094
18864
22270
23681
24333
25004
25734
26438
27229
28181
29783
31179
32089
33961
34490
35308
37387
38393
38893
40421
41348
42739
44279
45715
46737
51500
52373
53416
54025
55336
56086
58111
59615
61954
62412
63425
64967
66450
67651
69384
70677
71189
72243
73920
74343
75023
77376
78019
79352
80184
82410


KeyboardInterrupt: ignored

In [0]:
phrases_lp_stop.head()

In [0]:
windows_size = WINDOWS_SIZE
cont = 0
word_index = tokenizer
phrases_lp = pd.DataFrame(columns=['personId','answer', 't_answer'])
answers = all_participants_mix.groupby('personId').agg({'answer':'sum','t_answer':'sum'})

for p in answers.iterrows():      
    
    words = p[1]["answer"]
    size = len(words)
    word_tokens = p[1]["t_answer"]
    print(cont)
    for i in range(size):
        sentence = words[i:min(i+windows_size,size)]  
        tokens = word_tokens[i:min(i+windows_size,size)]  
        phrases_lp.loc[cont] = [p[0], sentence, tokens]
        cont = cont + 1

In [0]:
phrases_lp.to_csv('phrases_lp.csv', sep='\t')
print("File was created")

In [0]:
phrases_lp.head()

In [0]:
phrases_lp["t_answer"] = pad_sequences(phrases_lp["t_answer"], value=0, padding="post", maxlen=windows_size).tolist()
phrases_lp.head()

In [0]:
phrases_lp_stop.to_csv( 'phrases_lp_stop.csv', sep='\t')
print("File was created")

In [0]:
phrases_lp = pd.read_csv( 'phrases_lp.csv', sep='\t', converters={"t_answer": literal_eval}) 
phrases_lp.head()

In [0]:
def load_avec_dataset_file(path, score_column):
    ds = pd.read_csv(path, sep=',')
    ds['level'] = pd.cut(ds[score_column], bins=[-1,0,5,10,15,25], labels=[0,1,2,3,4])
    ds['PHQ8_Score'] = ds[score_column]
    ds['cat_level'] = keras.utils.to_categorical(ds['level'], num_classes).tolist()
    ds = ds[['Participant_ID', 'level', 'cat_level', 'PHQ8_Score']]
    ds = ds.astype({"Participant_ID": float, "level": int, 'PHQ8_Score': int})
    return ds

In [0]:
train = load_avec_dataset_file('train_split_Depression_AVEC2017.csv', 'PHQ8_Score')
dev = load_avec_dataset_file('dev_split_Depression_AVEC2017.csv', 'PHQ8_Score')
test = load_avec_dataset_file('full_test_split.csv', 'PHQ_Score')
print("Size: train= {}, dev= {}, test= {}".format(len(train), len(dev), len(test)))
train.head()

Size: train= 107, dev= 35, test= 47


,Participant_ID,level,cat_level,PHQ8_Score
0,303.0,0,"[1.0, 0.0, 0.0, 0.0, 0.0]",0
1,304.0,2,"[0.0, 0.0, 1.0, 0.0, 0.0]",6
2,305.0,2,"[0.0, 0.0, 1.0, 0.0, 0.0]",7
3,310.0,1,"[0.0, 1.0, 0.0, 0.0, 0.0]",4
4,312.0,1,"[0.0, 1.0, 0.0, 0.0, 0.0]",2


In [0]:
ds_total = pd.concat([train,dev,test])
total_phq8 = len(ds_total)
print("Total size = {}".format(total_phq8))

Total size = 189


In [0]:
ds_total.to_csv( 'ds_total.csv', sep = '\t')
print('done')

done


In [0]:
def split_by_phq_level(ds):
    none_ds = ds[ds['level']==0]
    mild_ds = ds[ds['level']==1]
    moderate_ds = ds[ds['level']==2]
    moderate_severe_ds = ds[ds['level']==3]
    severe_ds = ds[ds['level']==4]
    return (none_ds, mild_ds, moderate_ds, moderate_severe_ds, severe_ds)

In [0]:
none_ds, mild_ds, moderate_ds, moderate_severe_ds, severe_ds = split_by_phq_level(ds_total)
print("Quantity per none_ds: {}, mild_ds: {}, moderate_ds {}, moderate_severe_ds: {}, severe_ds {}".format(len(none_ds), len(mild_ds), len(moderate_ds), len(moderate_severe_ds), len(severe_ds)))

Quantity per none_ds: 26, mild_ds: 70, moderate_ds 47, moderate_severe_ds: 24, severe_ds 22


In [0]:
b_none_ds = ds_total[ds_total['level']==0]
b_mild_ds = ds_total[ds_total['level']==1].sample(26)
b_moderate_ds = ds_total[ds_total['level']==2].sample(26)
b_moderate_severe_ds = ds_total[ds_total['level']==3]
b_severe_ds = ds_total[ds_total['level']==4]

ds_total_b = pd.concat([b_none_ds, b_mild_ds, b_moderate_ds, b_moderate_severe_ds, b_severe_ds])
print("Quantity per none_ds: {}, mild_ds: {}, moderate_ds {}, moderate_severe_ds: {}, severe_ds {}".format(len(b_none_ds), len(b_mild_ds), len(b_moderate_ds), len(b_moderate_severe_ds), len(b_severe_ds)))

Quantity per none_ds: 26, mild_ds: 26, moderate_ds 26, moderate_severe_ds: 24, severe_ds 22


In [0]:
ds_lp = pd.merge(ds_total, phrases_lp,left_on='Participant_ID', right_on='personId')
ds_lp_b = pd.merge(ds_total_b, phrases_lp,left_on='Participant_ID', right_on='personId')

NameError: ignored

In [0]:
def distribute_instances(ds, split_in = [70,14,16]):
    ds_shuffled = ds.sample(frac=1)
    none_ds, mild_ds, moderate_ds, moderate_severe_ds, severe_ds = split_by_phq_level(ds_shuffled)
    eq_ds = dict()
    prev_none = prev_mild = prev_moderate = prev_moderate_severe = prev_severe = 0
    split = split_in
    for p in split:
        last_none = min(len(none_ds), prev_none + round(len(none_ds) * p/100))
        last_mild = min(len(mild_ds), prev_mild + round(len(mild_ds) * p/100))
        last_moderate = min(len(moderate_ds), prev_moderate + round(len(moderate_ds) * p/100))
        last_moderate_severe = min(len(moderate_severe_ds), prev_moderate_severe + round(len(moderate_severe_ds) * p/100))
        last_severe = min(len(severe_ds), prev_severe + round(len(severe_ds) * p/100))  
        eq_ds['d'+str(p)] = pd.concat([none_ds[prev_none: last_none], mild_ds[prev_mild: last_mild], moderate_ds[prev_moderate: last_moderate], moderate_severe_ds[prev_moderate_severe: last_moderate_severe], severe_ds[prev_severe: last_severe]])
        prev_none = last_none
        prev_mild = last_mild
        prev_moderate = last_moderate
        prev_moderate_severe = last_moderate_severe
        prev_severe = last_severe  
    return (eq_ds['d70'], eq_ds['d14'], eq_ds['d16'])

In [0]:
train_lp, dev_lp, test_lp = distribute_instances(ds_lp)
train_lp_b, dev_lp_b, test_lp_b = distribute_instances(ds_lp_b)

NameError: ignored

In [0]:
train_lp.tail()

NameError: ignored

In [0]:
def confusion_matrix(model, x, y):
    prediction = model.predict(x, batch_size=None, verbose=0, steps=None)
    labels=['none','mild','moderate','moderately severe', 'severe']

    max_prediction = np.argmax(prediction, axis=1)
    max_actual = np.argmax(y, axis=1)

    y_pred = pd.Categorical.from_codes(max_prediction, labels)
    y_actu = pd.Categorical.from_codes(max_actual, labels)

    return pd.crosstab(y_actu, y_pred)

In [0]:
embeddings_index = dict()
f = open('/content/drive/My Drive/glove/glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

FileNotFoundError: ignored

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def fill_embedding_matrix(tokenizer):
    vocab_size = len(tokenizer.word_index)
    embedding_matrix = np.zeros((vocab_size+1, 100))
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:        
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [0]:
embedding_matrix_lp = fill_embedding_matrix(tokenizer)

In [0]:
train_lp.head()

In [0]:
train_lp_copy = train_lp
print(len(train_lp_copy))
for index,row in train_lp_copy.iterrows():
    if len(row['t_answer'])<10:
        train_lp_copy.drop(index,inplace=True)
print(len(train_lp_copy))

In [0]:
train_lp = train_lp_copy

In [0]:
dev_lp_copy = dev_lp
print(len(dev_lp_copy))
for index,row in dev_lp_copy.iterrows():
    if len(row['t_answer'])<10:
        dev_lp_copy.drop(index,inplace=True)
print(len(dev_lp_copy))  
dev_lp = dev_lp_copy

In [0]:
test_lp_copy = test_lp
print(len(test_lp_copy))
for index,row in test_lp_copy.iterrows():
    if len(row['t_answer'])<10:
        test_lp_copy.drop(index,inplace=True)
print(len(test_lp_copy))  
test_lp = test_lp_copy

In [0]:
train_a = np.stack(train_lp['t_answer'], axis=0)
dev_a = np.stack(dev_lp['t_answer'], axis=0)
train_y = np.stack(train_lp['cat_level'], axis=0)
dev_y = np.stack(dev_lp['cat_level'], axis=0)

In [0]:
train_lp_b_copy = train_lp_b
print(len(train_lp_b_copy))
for index,row in train_lp_b_copy.iterrows():
    if len(row['t_answer'])<10:
        train_lp_b_copy.drop(index,inplace=True)
print(len(train_lp_b_copy))  
train_lp_b = train_lp_b_copy

In [0]:
dev_lp_b_copy = dev_lp_b
print(len(dev_lp_b_copy))
for index,row in dev_lp_b_copy.iterrows():
    if len(row['t_answer'])<10:
        dev_lp_b_copy.drop(index,inplace=True)
print(len(dev_lp_b_copy))  
dev_lp_b = dev_lp_b_copy

In [0]:
test_lp_b_copy = test_lp_b
print(len(test_lp_b_copy))
for index,row in test_lp_b_copy.iterrows():
    if len(row['t_answer'])<10:
        test_lp_b_copy.drop(index,inplace=True)
print(len(test_lp_b_copy))  
test_lp_b = test_lp_b_copy

In [0]:
train_a_b = np.stack(train_lp_b['t_answer'], axis=0)
dev_a_b = np.stack(dev_lp_b['t_answer'], axis=0)
train_y_b = np.stack(train_lp_b['cat_level'], axis=0)
dev_y_b = np.stack(dev_lp_b['cat_level'], axis=0)

In [0]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [0]:
answer_inp = Input(shape=(windows_size, ))
embedding_size_glove = 100
answer_emb1 = Embedding(vocab_size_stop+1, embedding_size_glove, weights=[embedding_matrix_lp], input_length=windows_size, trainable=False)(answer_inp)

bt = BatchNormalization()(answer_emb1)
lstm = LSTM(embedding_size_glove, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(bt)

dense1 = Dense(units=256, activation="relu")(lstm)
dense2 = Dense(units=256, activation="relu")(dense1)

flatten = Flatten()(dense2)

out = Dense(5,  activation='softmax')(flatten)

model = Model(inputs=[answer_inp], outputs=[out])
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [0]:
model_glove_lstm_hist = model.fit(train_a, train_y, validation_data=(dev_a, dev_y), epochs=30, batch_size=64, shuffle=True, callbacks=[early_stopping])

In [0]:
model_glove_lstm_hist_b = model.fit(train_a_b, train_y_b, validation_data=(dev_a_b, dev_y_b), epochs=30, batch_size=64, shuffle=True, callbacks=[early_stopping])

In [0]:
test_a = np.stack(test_lp['t_answer'], axis=0)
test_y = np.stack(test_lp['cat_level'], axis=0)
test_a_b = np.stack(test_lp_b['t_answer'], axis=0)
test_y_b = np.stack(test_lp_b['cat_level'], axis=0)
df_confusion = confusion_matrix(model, test_a_b, test_y_b)

In [0]:
df_confusion

In [0]:
model.save('/media/prahlad/New Volume/DIAC-WOZ/model_glove_lstm_b.h5')
json_dict = model_glove_lstm_hist.history
with open('/media/prahlad/New Volume/DIAC-WOZ/model_glove_lstm_b_hist.json', 'w') as f:
    f.write(str(json_dict))
print('File got created')

In [0]:
from keras.models import load_model
model = load_model('/media/prahlad/New Volume/DIAC-WOZ/model_glove_lstm_b.h5')

In [0]:
plot_loss(model_glove_lstm_hist)
plot_acc(model_glove_lstm_hist)
plot_loss(model_glove_lstm_hist_b)
plot_acc(model_glove_lstm_hist_b)

In [0]:
def test_model(text, model):
    print(text)
    word_list = text_to_wordlist(text)
    sequences = tokenizer.texts_to_sequences([word_list])
    sequences_input = list(itertools.chain(*sequences))
    sequences_input =  pad_sequences([sequences_input], value=0, padding="post", maxlen=windows_size).tolist()
    print(sequences_input)
    input_a = np.asarray(sequences_input)
    pred = model.predict(input_a, batch_size=None, verbose=0, steps=None)
    predicted_class = np.argmax(pred)
    print(labels[predicted_class])

In [0]:
sen = "All is going right with the party, I'm happy to know new people"
test_model(sen, model)
sen = "I want an ice cream and have some fries for lunch"
test_model(sen, model)
sen = "I'm afraid of losing my work, I don't have any money"
test_model(sen, model)
sen = "I'm worried about my future, I'm afraid of it"
test_model(sen, model)
sen = "I am a graduate student"
test_model(sen, model)
sen = "I am getting married"
test_model(sen, model)
sen = "This party is great, I know lots of people"
test_model(sen, model)
sen = "I miss my parents, brothers and sisters"
test_model(sen, model)
sen = "I detest my horrible job"
test_model(sen, model)
sen = "suicide"
test_model(sen, model)

In [0]:
answer_inp = Input(shape=(windows_size, ))
embedding_size_glove = 100
answer_emb1 = Embedding(vocab_size_stop+1, embedding_size_glove, weights=[embedding_matrix_lp], input_length=windows_size, trainable=False)(answer_inp)


lstm1 = LSTM(embedding_size_glove, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(answer_emb1)
lstm2 = LSTM(embedding_size_glove, dropout=0.2, recurrent_dropout=0.2)(lstm1)

X = Dropout(0.2)(lstm2)
bt = BatchNormalization()(X)
dense1 = Dense(units=256, activation="relu")(bt)

out = Dense(5,  activation='softmax')(dense1)

model_2lstm = Model(inputs=[answer_inp], outputs=[out])
model_2lstm.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
model_2lstm.summary()

In [0]:
model_glove_2lstm_b_hist = model_2lstm.fit(train_a_b, train_y_b, validation_data=(dev_a_b, dev_y_b), epochs=30, batch_size=64, shuffle=True, callbacks=[early_stopping])

In [0]:
plot_loss(model_glove_2lstm_b_hist)
plot_acc(model_glove_2lstm_b_hist)

In [0]:
score = model_2lstm.evaluate(test_a_b, test_y_b, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
df_confusion = confusion_matrix(model_2lstm, test_a_b, test_y_b)

In [0]:
df_confusion


In [0]:
model.save('/media/prahlad/New Volume/DIAC-WOZ/model_glove_2lstm_b.h5')
json_dict = model_glove_2lstm_b_hist.history
with open('/media/prahlad/New Volume/DIAC-WOZ/model_glove_2lstm_b_hist.json', 'w') as f:
    f.write(str(json_dict))
print('File got created')

In [0]:
sen = "All is going right with the party, I'm happy to know new people"
test_model(sen, model_2lstm)
sen = "I want an ice cream and have some fries for lunch"
test_model(sen, model_2lstm)
sen = "I'm afraid of losing my work, I don't have any money"
test_model(sen, model_2lstm)
sen = "I'm worried about my future, I'm afraid of it"
test_model(sen, model_2lstm)
sen = "My father loves me"
test_model(sen, model_2lstm)

In [0]:
sen = "Please kill me"
test_model(sen, model_2lstm)